In [ ]:
import numpy as np

from spin_operators import *

# helper functions numpy arrays

def iprint(M):
    print(M.astype(int))
    print()
    
def flat(*args): 
    for a in args:
        a.shape = (dim**2,)
        iprint(a)
        print()
    return None

def square(*args):
    for a in args:
        a.shape = (dim,dim)
        iprint(a)
        print()
    return None

def chop(M,eps=1e-9):
    
    w = np.where(np.abs(M) < eps)
    
    M[w] = 0
    
    print(M)
    
    print()


In regularity space operators are of the form:

$$\large
\mathrm{S}_{\ell}(\text{a},\text{b}) \ = \ \sum_{\sigma,\tau} \mathcal{Q}_{\ell}(\text{b},\sigma) S(\sigma,\tau) \mathcal{Q}_{\ell}(\text{a},\tau)
$$

$$\large
\mathrm{T}_{\ell}(\text{a},\text{b}) \ = \ \sum_{\sigma,\tau} \mathcal{Q}_{\ell}(\text{b},\sigma) T(\sigma,\tau) \mathcal{Q}_{\ell}(\text{a},\tau)
$$

The coeficient inbetween the $\mathcal{Q}$s are spin coupling coefficients. The $\ell$ dependence in regularity space makes everything much more annoying to work with. 

However, we can compose operators in spin space and then compute the regularity space action after the fact.

$$\large
(\mathrm{TS})_{\ell}(\text{a},\text{b}) \ = \ \sum_{\sigma,\tau} \mathcal{Q}_{\ell}(\text{b},\sigma) \left(\sum_{\kappa} T(\sigma,\kappa) S(\kappa,\tau) \right) \mathcal{Q}_{\ell}(\text{a},\tau)
$$

Being able to compose things allows us to build up any complicated thing in terms of elementary operations in spin space. 

As far as I can see the elementary operations are 

* Trace over any two tensor indices. 



* Transpose (or permutation) over any collection of indices.



* Tensor product with any tensorial spin basis element.



* Multiplication by the identity.



* Multiplication by the metric (physical space identity).



* Multiplication be the completly antisymmetric tensor of rank-3.




* The linear combination of compatible (same input and output) operators.



* The adjoint (transpose) of any of the above operators (reversing input and output). 



* The composition of any compatible operators.

In [ ]:
spins  = (-1,0,+1)
dim    = len(spins)

I  = Identity(indexing=spins)

M  = Metric(indexing=spins)

T  = Transpose(permutation=(1,0),indexing=spins)

Tr = Trace(indices=(0,1),indexing=spins)

P  = lambda kappa: TensorProduct(kappa,indexing=spins)

Q = lambda ell: Intertwiner(ell,indexing=spins)

Start with a flattened rank-2 array. Apply the transpose operator and look at the square results:

In [ ]:
A = np.arange(dim**2,dtype=float)

iprint(A)

In [ ]:
B = T(2) @ A

iprint(B)

square(A,B)

The rank-2 transpose operator matrix

In [ ]:
iprint(T(2))

The trace in spin space is 

$$
\text{Tr}(A) = A(-1,+1) + A(0,0) + A(+1,-1)
$$

In [ ]:
square(A)

if dim == 3: iprint(A[0,2] + A[2,0] + A[1,1])
if dim == 2: iprint(A[0,1] + A[1,0])

The metric moves the cross diagonal into the diagonal

In [ ]:
iprint(M(1))
iprint(A)
iprint( M(1) @ A )
iprint(np.trace(M(1) @ A))

Re-flatten `A` and take the trace operator

In [ ]:
Tr.indices

In [ ]:
flat(A)

iprint(Tr(2) @ A)

The rank-2 trace operator matrix

In [ ]:
iprint(Tr(2))

The non-zeros spin elements:

In [ ]:
print(Tr[(),(-1,+1)])
print(Tr[(),( 0, 0)])
print(Tr[(),(+1,-1)])

The trace is a left eigenvector of transpose.

In [ ]:
iprint(Tr(2) @ T(2) - Tr(2))

This also works constructing the same thing as abstract operators.

In [ ]:
Z = Tr @ T - Tr

iprint(Z(2))

We can construct the cotrace operator as the transpose.

In [ ]:
Cotr = Tr.T

iprint(Cotr(0))

The square operator multiplies by the dimension:

In [ ]:
iprint((Tr @ Cotr)(0))
iprint((Tr @ Cotr)(1))
iprint((Tr @ Cotr)(2))

What does this do?

In [ ]:
S = I + T - Cotr @ Tr 

iprint(S(2))

In [ ]:
Tr2 = Cotr @ Tr 
iprint(Tr2(2))

Evaluation keeps track of the rank

In [ ]:
iprint((Cotr @ Tr)(2) - Cotr(0) @ Tr(2))

Same with acting against arrays

What is the `Cotr @ Tr` operator?

Here is the Metric

In [ ]:
iprint(M(1))
iprint(M(2))

In [ ]:
B = (Cotr @ Tr)(2) @ A
iprint(M(1)*(Tr(2) @ A))
iprint((Tr(2) @ A)*M(1))
square(B)

The `Cotr @ Tr` operator multiplies the trace times the metric identity.

We can interact with the spin elements directly:

In [ ]:
print(T[(-1,+1),(+1,-1)])
print(Tr[(),(+1,-1)])
print(Tr[(),(+1,+1)])

The tensor product depends on the spin basis element.

Here is a spin e(1) acting on a rank-k field (scalar,vector,2-tensor):

$$\large
e(1) \otimes T \quad \text{as an abstract operator}
$$

In [ ]:
iprint(P(+1)(0))
iprint(P(+1)(1))
iprint(P(+1)(2))

We can linearly combine tensor products over basis elements:

In [ ]:
S = 7*P(-1) + 3*P(0) - 2*P(1)

iprint(S(0))
iprint(S(1))
iprint(S(2))

We can see that `S` increases the rank of anything by one.

In [ ]:
S.codomain

For any input with rank > 0 we can contract over the (0,1) indices with `Tr`

$$\large
(7 e(-) + 3 e(0) - 2 e(+)) \cdot T
$$

In [ ]:
(Tr @ S).codomain

In [ ]:
iprint((Tr @ S)(1))
iprint((Tr @ S)(2))

For 2 < rank we can contract twice:

$$\large
\text{Tr} \left( (7 e(-) + 3 e(0) - 2 e(+)) \cdot T \right)
$$

In [ ]:
print((Tr @ Tr @ S).codomain)
iprint((Tr @ Tr @ S)(3))

In general we ca use different trace operators to contract over different indices:

In [ ]:
Tr = lambda i,j: Trace(indices=(i,j),indexing=spins)

In [ ]:
iprint((Tr(0,1) @ Tr(0,1) @ S)(3))
iprint((Tr(0,1) @ Tr(0,2) @ S)(3))
iprint((Tr(0,1) @ Tr(0,3) @ S)(3))
iprint((Tr(0,1) @ Tr(1,2) @ S)(3))
iprint((Tr(0,1) @ Tr(2,3) @ S)(3))
iprint((Tr(0,1) @ Tr(1,3) @ S)(3))

Thinking of $\mathrm{V}$ as the tensor-product operator, and $T$ as the rank-3 tensor input, these matrices represent the following respective actions:

$$
\sum_{\sigma,\tau} \mathrm{V}_{\sigma}\ T_{-\sigma,\,\tau,\,-\tau}
$$

$$
\sum_{\sigma,\tau} \mathrm{V}_{\sigma}\ T_{\tau,\,-\sigma,\,-\tau}
$$

$$
\sum_{\sigma,\tau} \mathrm{V}_{\sigma}\ T_{\tau,\,-\tau,\,-\sigma}
$$

$$
\sum_{\sigma,\tau} \mathrm{V}_{\tau}\ T_{\sigma,\,-\sigma,\,-\tau}
$$

$$
\sum_{\sigma,\tau} \mathrm{V}_{\tau}\ T_{\sigma,\,-\tau,\,-\sigma}
$$

$$
\sum_{\sigma,\tau} \mathrm{V}_{\tau}\ T_{-\tau\,\,\sigma,\,-\sigma}
$$

Because we can reverse the order of the sums:

In [ ]:
rank = 3

Z1 = Tr(0,1) @ (Tr(0,1)-Tr(2,3)) @ S
Z2 = Tr(0,1) @ (Tr(0,2)-Tr(1,3)) @ S
Z3 = Tr(0,1) @ (Tr(0,3)-Tr(1,2)) @ S
iprint(Z1(rank))
iprint(Z2(rank))
iprint(Z3(rank))

For rank-4;

In [ ]:
rank = 4
iprint(Z1(rank))
iprint(Z2(rank))
iprint(Z3(rank))

Last we have the spin-to-regularity intertwiners.

For rank=1:

In [ ]:
for ell in range(5):
    chop(Q(ell)(1))
    print()

Calling on rank=2:

In [ ]:
for ell in range(4):
    chop(Q(ell)(2))
    print()

We can also index the elements directly:

In [ ]:
spin = 0
regularity = 1

Q(0)[spin,regularity]


spin = (-1,1)
regularity = (0,1)

Q(0)[spin,regularity]

We can see the operator is orthogonal

In [ ]:
for ell in range(4):

    chop((Q(ell).T @ Q(ell))(1))

    chop((Q(ell) @ Q(ell).T)(1))

    print()

In [ ]:
for ell in range(3):

    chop((Q(ell).T @ Q(ell))(2))

    chop((Q(ell) @ Q(ell).T)(2))

    print()


We can build regularity-space operators out of the spin representations

For the metric:

In [ ]:
for ell in range(4):
    chop((Q(ell).T @ M @ Q(ell))(1))

For the trace:

In [ ]:
Tr = Tr(0,1)

In [ ]:
for ell in range(4):
    chop((Q(ell).T @ Tr @ Q(ell))(2))

I'll have to check the individual entries against analytical formulae. IDK if it's perfect. But it's generally working right.

In [ ]:
TRACE = np.zeros((3,3))

for i,a in enumerate((-1,0,1)):
    for j,b in enumerate((-1,0,1)):

        TRACE[i,j] = (Q(0).T @ Tr @ Q(0))[(),(a,b)]
        
chop(TRACE)

It seems that the value is correct. But it might be the wrong place.

In [ ]:
print((Q(0).T @ Tr @ Q(0))[(),(-1,1)]**2)